# 00 — Data Ingestion (Step-by-step)
**Goal:** Walk through using `get_market_data` and `get_options_data` with caching.

---


In [1]:
# 1. Environment setup
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from datetime import date, timedelta
from oami.config import initialize_environment
from oami.data_layer import get_market_data, get_options_data


api_key = initialize_environment()
print('API key configured' if api_key != 'YOUR_KEY_HERE' else 'Running in offline mode using cache')


## 2. Choose symbol and date window
We'll work with `SPY` and fetch the last 30 days of data.


In [2]:
symbol = 'SPY'
end_date = date.today()
start_date = end_date - timedelta(days=30)
print(f'Requesting data from {start_date} to {end_date}')


## 3. Fetch market data
`get_market_data` re-uses cached CSVs in `data/csv/market/<interval>/<symbol>.csv`. Missing ranges are pulled from Polygon and persisted automatically.


In [3]:
market_df = get_market_data(
    symbol=symbol,
    start=start_date.isoformat(),
    end=end_date.isoformat(),
    interval='1D',
    use_cache=True,
)
print('Market rows:', len(market_df))
market_df.head()


## 4. Fetch options data
`get_options_data` pulls contract metadata, stores per-contract aggregates in `data/options`, then builds a daily sentiment summary.


In [ ]:
options_df = get_options_data(
    symbol=symbol,
    start_date=start_date.isoformat(),
    end_date=end_date.isoformat(),
    interval='1D',
    use_cache=True,
)
print('Options rows:', len(options_df))
options_df.head()


## 5. Where did the data go?
- Market cache: `data/csv/market/1D/{symbol}.csv`
- Contracts: `data/options/{symbol}/contracts/<YYYY-MM-DD>.csv`
- Aggregates per contract: `data/options/{symbol}/1day/<expiry>/<contract>.csv`
- Options summary: `data/csv/options/1D/{symbol}.csv`

Re-running the notebook with the same window now reloads directly from these caches.
